In [198]:
with open('topo.txt', 'r') as f:
    topo_raw = f.read()

In [199]:
from itertools import product

In [200]:
product()

In [201]:
list()

[]

In [202]:
ord('a')

97

In [203]:
# build topo graph

class Node(object):
    NODES = {}
    START = None
    END = None
    
    UNCHECKED = set()
    
    _ZERO_OFFSET = -ord('a')
    
    def __init__(self, x, y, h):
        self.x = x
        self.y = y
        
        self._dist = None
        
        if h == 'S':
            Node.START = self
            self.h = ord('a') + Node._ZERO_OFFSET
        elif h == 'E':
            Node.END = self
            self.h = ord('z') + Node._ZERO_OFFSET
        else:
            self.h = ord(h) + Node._ZERO_OFFSET
        
        Node.NODES[(x,y)] = self
        
        self._exits = None
        
    
    @property
    def dist(self):
        return self._dist
    
    @dist.setter
    def dist(self, new_value):
        if self._dist is not None:
            return # don't do anything if already set
        self._dist = new_value
        self.UNCHECKED.remove(self)
        if self is self.END:
            raise KeyboardInterrupt
        
    @classmethod
    def reset_all_dist(cls):
        cls.UNCHECKED = set(cls.NODES.values())
        for node in cls.NODES.values():
            node._dist = None
        cls.START.dist = 0
    
    
    @property
    def exits(self):
        if self._exits is None:
            self._exits = set()
            for x_offset, y_offset in [(-1,0), (1,0), (0,-1), (0,1)]:
                try:
                    node = Node.NODES[(self.x+x_offset, self.y+y_offset)]
                except KeyError:
                    continue
                if -1 <= node.h - self.h <= 1:
                    self._exits.add(node)
        return self._exits
    
    
    def deadend(self, exclusions):
        return not bool(self.exits - exclusions)
    

    def __repr__(self):
        return '[%2d %2d] %2d %s%s' % (self.x, self.y, self.h, chr(self.h + ord('a')), ' (%d from start)' % self._dist if self._dist else '')

In [204]:
for y, line in enumerate(topo_raw.splitlines()):
    for x, h in enumerate(line):
        _ = Node(x,y,h)
        
for node in Node.NODES.values():
    _ = node.exits

In [205]:
Node.START

[ 0 20]  0 a

In [206]:
Node.START.exits

{[ 0 19]  0 a, [ 0 21]  0 a, [ 1 20]  1 b}

# Brute force didn't work, try one-shot exploration

So spread a wave-front from origin. If the number of steps is set, don't follow it since it was reached earlier. If it's unset, merely set the step.



In [207]:
from time import sleep
from IPython.display import clear_output

In [212]:
Node.reset_all_dist()

front = set([Node.START])

for step in range(1,100000):
    clear_output(wait=True)
    print_topo(front)
    sleep(0.2)

    exits = set(exit
                for node in front
                for exit in node.exits
                if exit in Node.UNCHECKED
               )
    
    if not exits:
        raise RuntimeError('Graph dried up at step %d' %step)
    
    for exit in exits:
        exit.dist = step
            
    front = exits


.......aaaaaa...aaa..aaaaaaaa...aaaaaa............................aaaaaaaaaaaaaaca...............................aaaaaaaa.............................................aaaaa
.......aaaaaaaa.aaaa..aaaaaa....aaaaaa..........aaa...............aaaaaaaaaaaaaaaa...............................aaaaaaaaa......aaa..................................aaaaaa
......aaaaaaaaa.aaaa..aaaaaa....aaaaaaaa........aaa................aaaaaaaaaaaaaa....aaa..........................aaaaaaaa....a.aaa................aa................aaaaaa
.....aaaaaaaaaa.aaa..aaaaaaaa.....aa..aaa.aa....aaaaaaa............aacaaaaaaaaa......aaaa....................aaa.aaaaaa......aaaaaaaa.............aaaa...............aaa.aa
.....aaa.aaa........aaaaaaaaaa........aaaaaa.aaaaaaaaaa..............aaaaaaaaaa......aaaa................aa..aaa.aaaaaaa.....aaaaaaaa.............aaaa..aaa..............aa
.........aaa........aaaaaaaaaa......aaaaaaa..aaaaaaaaa...............aaaa.aaaaaaaa...aaaa................aaaaaaa.aa..aaa.......aaaaa........

KeyboardInterrupt: 

In [210]:
Node.END

[146 20] 25 z (520 from start)

# PART TWO: RUN IT BACKWARDS

In [219]:
# build topo graph

class Node(object):
    NODES = {}
    START = None
    
    UNCHECKED = set()
    
    _ZERO_OFFSET = -ord('a')
    
    def __init__(self, x, y, h):
        self.x = x
        self.y = y
        
        self._dist = None
        
#         if h == 'S':
#             Node.START = self
#             self.h = ord('a') + Node._ZERO_OFFSET
        if h == 'E':
            Node.START = self
            self.h = ord('z') + Node._ZERO_OFFSET
        else:
            self.h = ord(h) + Node._ZERO_OFFSET
        
        Node.NODES[(x,y)] = self
        
        self._exits = None
        
    
    @property
    def dist(self):
        return self._dist
    
    @dist.setter
    def dist(self, new_value):
        if self._dist is not None:
            return # don't do anything if already set
        self._dist = new_value
        self.UNCHECKED.remove(self)
        if self.h == 0:
            raise KeyboardInterrupt
        
    @classmethod
    def reset_all_dist(cls):
        cls.UNCHECKED = set(cls.NODES.values())
        for node in cls.NODES.values():
            node._dist = None
        cls.START.dist = 0
    
    
    @property
    def exits(self):
        if self._exits is None:
            self._exits = set()
            for x_offset, y_offset in [(-1,0), (1,0), (0,-1), (0,1)]:
                try:
                    node = Node.NODES[(self.x+x_offset, self.y+y_offset)]
                except KeyError:
                    continue
                if -1 <= node.h - self.h <= 1:
                    self._exits.add(node)
        return self._exits
    
    
    def deadend(self, exclusions):
        return not bool(self.exits - exclusions)
    

    def __repr__(self):
        return '[%2d %2d] %2d %s%s' % (self.x, self.y, self.h, chr(self.h + ord('a')), ' (%d from start)' % self._dist if self._dist else '')

In [220]:
for y, line in enumerate(topo_raw.splitlines()):
    for x, h in enumerate(line):
        _ = Node(x,y,h)
        
for node in Node.NODES.values():
    _ = node.exits

In [221]:
Node.reset_all_dist()

front = set([Node.START])

for step in range(1,100000):
    clear_output(wait=True)
    print_topo(front)
    sleep(0.2)

    exits = set(exit
                for node in front
                for exit in node.exits
                if exit in Node.UNCHECKED
               )
    
    if not exits:
        raise RuntimeError('Graph dried up at step %d' %step)
    
    for exit in exits:
        exit.dist = step
            
    front = exits


abcccccaaaaaa...aaa..aaaaaaaa...aaaaaa............................aaaaaaaaaaaaaaca...............................aaaaaaaa.............................................aaaaa
abcccccaaaaaaaa.aaaa..aaaaaa....aaaaaa..........aaa...............aaaaaaaaaaaaaaaa...............................aaaaaaaaa......aaa..................................aaaaaa
abccccaaaaaaaaa.aaaa..aaaaaa....aaaaaaaa........aaa................aaaaaaaaaaaaaa....aaa..........................aaaaaaaa....a.aaa................aa................aaaaaa
abcc aaaaaaaaaa.aaa..aaaaaaaa.....aa..aaa.aa....aaaaaaa............aacaaaaaaaaa......aaaa....................aaa.aaaaaa......aaaaaaaa.............aaaa...............aaa.aa
abc .aaa.aaa........aaaaaaaaaa........aaaaaa.aaaaaaaaaa..............aaaaaaaaaa......aaaa................aa..aaa.aaaaaaa.....aaaaaaaa.............aaaa..aaa..............aa
ab ......aaa........aaaaaaaaaa......aaaaaaa..aaaaaaaaa...............aaaa.aaaaaaaa...aaaa................aaaaaaa.aa..aaa.......aaaaa........

KeyboardInterrupt: 

In [222]:
step

508

# Old nonsense work while I was both feeling out the problem and then discovering the bug in my input

In [209]:
Node.START

[ 0 20]  0 a

In [136]:
front

{[162 24] 12 m (358 from start)}

In [137]:
front

{[162 24] 12 m (358 from start)}

In [138]:
exits

set()

In [186]:
def print_topo(highlight=set()):
    line = []
    n = 0
    for x,y in sorted(Node.NODES, key=lambda e: (e[1], e[0])):
        if y != n:
            print(''.join(line))
            line = []
            n += 1
        node = Node.NODES[(x,y)]
        line.append(
                ' ' if node in highlight else # interactive!
                chr(ord('a') + node.h) if not node.dist # pending
                else '.' # already visited
        )
    else:
        print(''.join(line))


In [187]:
print_topo()

.......aaaaaa...aaa..aaaaaaaacccaaaaaaccccccccccccccccccccccccccccaaaaaaaaaaaaaacacccccccccccccccccccccccccccccccaaaaaaaacccccccccccccccccccccccccccccccccccccccccccccaaaaa
.......aaaaaaaa.aaaa..aaaaaaccccaaaaaaccccccccccaaacccccccccccccccaaaaaaaaaaaaaaaacccccccccccccccccccccccccccccccaaaaaaaaaccccccaaaccccccccccccccccccccccccccccccccccaaaaaa
......aaaaaaaaa.aaaa..aaaaaaccccaaaaaaaaccccccccaaaccccccccccccccccaaaaaaaaaaaaaaccccaaaccccccccccccccccccccccccccaaaaaaaaccccacaaaccccccccccccccccaaccccccccccccccccaaaaaa
.....aaaaaaaaaa.aaa..aaaaaaaacccccaaccaaacaaccccaaaaaaaccccccccccccaacaaaaaaaaaccccccaaaaccccccccccccccccccccaaacaaaaaaccccccaaaaaaaacccccccccccccaaaacccccccccccccccaaacaa
.....aaa.aaa........aaaaaaaaaaccccccccaaaaaacaaaaaaaaaaccccccccccccccaaaaaaaaaaccccccaaaaccccccccccccccccaaccaaacaaaaaaacccccaaaaaaaacccccccccccccaaaaccaaaccccccccccccccaa
.........aaa........aaaaaaaaaaccccccaaaaaaaccaaaaaaaaacccccccccccccccaaaacaaaaaaaacccaaaaccccccccccccccccaaaaaaacaaccaaacccccccaaaaacccccccc

In [86]:
Node.reset_all_dist()

front = set([Node.START])

for step in range(10000):
    assert front
    
    
    exits = set(exit
                for node in front
                for exit in node.exits
               )
    exits -= Node.CHECKED
    
    for exit in exits:
        exit.dist = step
    
    front = exits
else:
    raise RuntimeError('Ran out of loops')

AssertionError: 

In [88]:
step

359

In [90]:
Node.END.exits

{[147 20] 25}

In [67]:
Node.reset_all_dist()

front = set([Node.START])

In [70]:
set(exit
    for node in front
    for exit in node.exits
   ) - Node.CHECKED

{[ 0 19]  0, [ 0 21]  0, [ 1 20]  1}

In [66]:
Node.START.exits

{[ 0 19]  0, [ 0 21]  0, [ 1 20]  1}

In [55]:
step = 0
this = set()
last = set()

Node.reset_all_dist()

last.add(Node.START)

while True:
    step += 1
    
    this = set(exit 
                for node in last
                for exit in node.exits
                if not exit.dist
              )
        
    for node in this:
        node.dist = step
        if node is Node.END:
            raise StopIteration
        
    last = this

KeyboardInterrupt: 

In [58]:
Node.END.exits

{[147 20] 25}

In [57]:
step

7012617

# Naive approach
Check paths. This is Big O bad

In [11]:
def walk(node, visited, paths):
    
    if node is Node.END:
        paths.append(frozenset(visited))
        return
    
    if node.deadend(visited):
        return
    
    for exit in node.exits:
        if exit in visited:
            continue
        visited.add(exit)
        walk(exit, visited, paths)
        visited.remove(exit)

In [12]:
# results = []
# walk(Node.START, set([Node.START]), results)

In [13]:
len(results)

NameError: name 'results' is not defined